In [125]:
#install.packages(c("tree", "caret", "rpart", "rpart.plot","datasets"))
#install.packages(c( "ggplot2", "datasets","ISLR","randomForest"))
# install.packages(c("lava", "prodlim", "ipred", "ModelMetrics", "foreach", "iterators", "timeDate"), type = "binary")

In [126]:
# cambio al directorio
#output_dir <- "C:/Users/user2/Documents/GitHub/Decision_Trees/r/output"

In [127]:
library(tree)
library(caret)
library(rpart)
library(rpart.plot)
library(ggplot2)

## 1.1 Limpieza de Data

### Step 1: Load the Data

In [128]:
# Read the data (adjust the path to your data location)
url <- "C:/Users/user2/Documents/GitHub/Decision_Trees/r/input/processed.cleveland.data"
df <- read.csv(url, header = FALSE, na.strings = "?")

head(df)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
2,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
3,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
4,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
5,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0
6,56,1,2,120,236,0,0,178,0,0.8,1,0,3,0


### Step 2: Rename Variables

In [129]:
# Rename the columns
colnames(df) <- c('age', 'sex', 'cp', 'restbp', 'chol', 'fbs', 'restecg', 
                  'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'hd')

# Display column names
cat("Column names:\n")
print(colnames(df))

# Display structure
str(df)

Column names:
 [1] "age"     "sex"     "cp"      "restbp"  "chol"    "fbs"     "restecg"
 [8] "thalach" "exang"   "oldpeak" "slope"   "ca"      "thal"    "hd"     
'data.frame':	303 obs. of  14 variables:
 $ age    : num  63 67 67 37 41 56 62 57 63 53 ...
 $ sex    : num  1 1 1 1 0 1 0 0 1 1 ...
 $ cp     : num  1 4 4 3 2 2 4 4 4 4 ...
 $ restbp : num  145 160 120 130 130 120 140 120 130 140 ...
 $ chol   : num  233 286 229 250 204 236 268 354 254 203 ...
 $ fbs    : num  1 0 0 0 0 0 0 0 0 1 ...
 $ restecg: num  2 2 2 0 2 0 2 0 2 2 ...
 $ thalach: num  150 108 129 187 172 178 160 163 147 155 ...
 $ exang  : num  0 1 1 0 0 0 0 1 0 1 ...
 $ oldpeak: num  2.3 1.5 2.6 3.5 1.4 0.8 3.6 0.6 1.4 3.1 ...
 $ slope  : num  3 2 2 3 1 1 3 1 2 3 ...
 $ ca     : num  0 3 2 0 0 0 2 0 1 0 ...
 $ thal   : num  6 3 7 3 3 3 3 3 7 7 ...
 $ hd     : int  0 2 1 0 0 0 3 0 2 1 ...


### Step 3: Remove Missing Values

In [130]:
# Check for missing values
cat("Missing values per column:\n")
print(colSums(is.na(df)))

cat("\nTotal rows antes removing NA:", nrow(df), "\n")

# Remove rows with missing values
df <- na.omit(df)

cat("Total rows despues removing NA:", nrow(df), "\n")

Missing values per column:
    age     sex      cp  restbp    chol     fbs restecg thalach   exang oldpeak 
      0       0       0       0       0       0       0       0       0       0 
  slope      ca    thal      hd 
      0       4       2       0 

Total rows antes removing NA: 303 
Total rows despues removing NA: 297 


### Step 4: Create Binary Target Variable y

In [131]:
# Create binary target variable
df$y <- ifelse(df$hd > 0, 1, 0)

# Convert to factor for classification
df$y <- as.factor(df$y)

# Display the distribution
cat("Distribution of target variable y:\n")
print(table(df$y))
cat("\nProportions:\n")
print(prop.table(table(df$y)))

Distribution of target variable y:

  0   1 
160 137 

Proportions:

        0         1 
0.5387205 0.4612795 


In [132]:
# conversion de varaible categoricas
categorical_vars <- c('sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal')

for (var in categorical_vars) {
    df[[var]] <- as.factor(df[[var]])
}

#str(df)

In [133]:
var_name <- c('age', 'sex', 'cp', 'restbp', 'chol', 'fbs', 'restecg', 
                  'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'y')

df_model <- df[, var_name]

formula_str <- paste("~ . - y")

# Creo dummys
X_matrix <- model.matrix(as.formula(formula_str), data = df_model)[, -1]  # quito intercepto
df_final <- as.data.frame(X_matrix)
df_final$y <- df_model$y

head(df_final)

,age,sex1,cp2,cp3,cp4,restbp,chol,fbs1,restecg1,restecg2,⋯,exang1,oldpeak,slope2,slope3,ca1,ca2,ca3,thal6,thal7,y
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,63,1,0,0,0,145,233,1,0,1,⋯,0,2.3,0,1,0,0,0,1,0,0
2,67,1,0,0,1,160,286,0,0,1,⋯,1,1.5,1,0,0,0,1,0,0,1
3,67,1,0,0,1,120,229,0,0,1,⋯,1,2.6,1,0,0,1,0,0,1,1
4,37,1,0,1,0,130,250,0,0,0,⋯,0,3.5,0,1,0,0,0,0,0,0
5,41,0,1,0,0,130,204,0,0,1,⋯,0,1.4,0,0,0,0,0,0,0,0
6,56,1,1,0,0,120,236,0,0,0,⋯,0,0.8,0,0,0,0,0,0,0,0


## 1.2 Analisis de Data

### 1.2.1 Split Data and Plot Classification Tree (1 point)

In [134]:
#semilla
set.seed(123)

#  training (70%) y testing (30%), creo partciiones estratificadas
train_index <- createDataPartition(df_final$y, p = 0.7, list = FALSE)
train_data <- df_final[train_index, ]
test_data <- df_final[-train_index, ]

cat("tamaño Training:", nrow(train_data), "\n")
cat("tamaño Testing:", nrow(test_data), "\n")

tamaño Training: 208 
tamaño Testing: 89 


In [135]:
# Train classification tree
set.seed(123)
tree_model <- rpart(y ~ ., data = train_data, method = "class", 
                    control = rpart.control( cp = 0 ,minsplit = 2,minbucket = 1      )) # rpart.control() maxdepth = 3

# Plot the tree
png(paste0(output_dir, "/tree_antes_pruning.png"), width = 1200, height = 800, res = 120)

rpart.plot(tree_model,             
           extra = 101,      
           box.palette = c("pink", "lightblue"),  
           shadow.col = "gray", 
           under = TRUE,         
           faclen = 0,            
           cex = 0.6,             
           main = "Classification Tree - Heart Disease (antes Pruning)")
dev.off()

pdf 
  2

### 1.2.2 Plot Confusion Matrix and Interpret (2 points)

In [136]:
y_pred <- predict(tree_model, newdata = test_data, type = "class")

#  confussion matrix
cm <- confusionMatrix(y_pred, test_data$y, positive = "1")
print(cm)

Confusion Matrix and Statistics

          Reference
Prediction  0  1
         0 32 14
         1 16 27
                                          
               Accuracy : 0.6629          
                 95% CI : (0.5549, 0.7597)
    No Information Rate : 0.5393          
    P-Value [Acc > NIR] : 0.01213         
                                          
                  Kappa : 0.3241          
                                          
 Mcnemar's Test P-Value : 0.85513         
                                          
            Sensitivity : 0.6585          
            Specificity : 0.6667          
         Pos Pred Value : 0.6279          
         Neg Pred Value : 0.6957          
             Prevalence : 0.4607          
         Detection Rate : 0.3034          
   Detection Prevalence : 0.4831          
      Balanced Accuracy : 0.6626          
                                          
       'Positive' Class : 1               
                                    

In [137]:
# graficar confusion matrix
cm_table <- as.data.frame(table(Predicted = y_pred, Actual = test_data$y))

labels <- c("0" = "Does not have HD", "1" = "Has HD")
cm_table$Predicted <- factor(cm_table$Predicted, levels = c("0", "1"), labels = labels)
cm_table$Actual <- factor(cm_table$Actual, levels = c("0", "1"), labels = labels)

p1 <- ggplot(cm_table, aes(x = Actual, y = Predicted, fill = Freq)) +
  geom_tile(color = "white") +
  geom_text(aes(label = Freq), size = 8, color = "white") +
  scale_fill_gradient(low = "lightblue", high = "darkblue") +
  labs(title = "Confusion Matrix - antes Pruning",
       x = "Actual",
       y = "Predicted") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

ggsave(
  filename = paste0(output_dir, "/confusion_matrix_antes_pruning.png"),
  plot = p1,  
  width = 8,
  height = 6,
  dpi = 300
)

**Interpretacion:**

Los valores que estan en la columna secundaria de azul son los valores predichos correctos (se puede ver en la imagen "confusion_marix_antes_prunnin.png"). En este caso, 31 fueron detectados correctamente sanos y 31 tambien fueron detectados como enfermos. Mientras que 15 fueron detectados como enfermos cuando estaban sanos y 13 fueron detectados como sanos cuando estaban enfermos del corazón

### 1.2.3 Fix Overfitting with Cross-Validation (1.5 points)

In [138]:
# 50 alphas desde  e^-10 a 0.05
alpha_min <- exp(-10)
alpha_max <- 0.05
alphas <- exp(seq(log(alpha_min), log(alpha_max), length.out = 50))

In [139]:
set.seed(123)
folds <- createFolds(train_data$y, k = 4, list = TRUE, returnTrain = FALSE)

cv_accuracy <- function(alpha, data, folds) {
  accuracies <- numeric(length(folds))
  
  for (i in seq_along(folds)) {
    # split de data
    val_idx <- folds[[i]]
    cv_train <- data[-val_idx, ]
    cv_val <- data[val_idx, ]
    
    # entrenamiento modelo por cada alpha
    model <- rpart(y ~ ., data = cv_train, method = "class",
                   control = rpart.control(cp = alpha, minsplit = 2, minbucket = 1))
    
    pred <- predict(model, newdata = cv_val, type = "class")
    accuracies[i] <- mean(pred == cv_val$y)
  }
  
  return(mean(accuracies))
}

# Calculo de CV por cada alpha
cv_scores <- sapply(alphas, function(a) cv_accuracy(a, train_data, folds))

In [140]:
# alpha optimo
best_idx <- which.max(cv_scores)
best_alpha <- alphas[best_idx]
best_accuracy <- cv_scores[best_idx]

cat("alpha optimo :", best_alpha, "\n")
cat("mejor CV:", best_accuracy, "\n")

alpha optimo : 0.02822607 
mejor CV: 0.7548077 


### 1.2.4 Plot Inaccuracy Rate vs Alpha (1.5 points)

In [141]:
#ratio de inaccuracy
inaccuracy_rates <- 1 - cv_scores
plot_data <- data.frame(alpha = alphas, inaccuracy = inaccuracy_rates)

p2 <-ggplot(plot_data, aes(x = alpha, y = inaccuracy)) +
  geom_line(color = "blue", size = 1) +
  geom_point(color = "blue", size = 2) +
  geom_vline(xintercept = best_alpha, color = "red", linetype = "dashed", size = 1) +
  annotate("text", x = best_alpha * 1.5, y = max(inaccuracy_rates) * 0.9,
           label = paste("Optimal α =", round(best_alpha, 6)),
           color = "red", size = 4) +
  scale_x_log10() +
  labs(title = "Inaccuracy Rate vs Alpha (4-Fold Cross-Validation)",
       x = "Alpha (log scale)",
       y = "Inaccuracy Rate (1 - Accuracy)") +
  theme_minimal() +
  theme(plot.title = element_text(hjust = 0.5))


 ggsave(
  filename = paste0(output_dir, "/inaccuracy_rate.png"),
  plot = p2,  
  width = 8,
  height = 6,
  dpi = 300
) 


### 1.2.5 Plot Pruned Tree and Confusion Matrix (2 points)

In [142]:
# tree con alpha óptimo
png(paste0(output_dir, "/tree_despues_pruning.png"), width = 1200, height = 800, res = 120)
pruned_tree <- rpart(y ~ ., data = train_data, method = "class",
                     control = rpart.control(cp = best_alpha, minsplit = 2, minbucket = 1))

rpart.plot(pruned_tree,
           type = 4,
           extra = 101,
           under = TRUE,
           faclen = 0,
           cex = 0.8,
           main = paste("Classification Tree - Heart Disease (Pruned, α =", round(best_alpha, 6), ")"))

dev.off()

pdf 
  2

In [143]:
y_pred_pruned <- predict(pruned_tree, newdata = test_data, type = "class")

# matrix confusion
cm_pruned <- confusionMatrix(y_pred_pruned, test_data$y, positive = "1")

print(cm_pruned)

Confusion Matrix and Statistics

          Reference
Prediction  0  1
         0 33 10
         1 15 31
                                          
               Accuracy : 0.7191          
                 95% CI : (0.6138, 0.8093)
    No Information Rate : 0.5393          
    P-Value [Acc > NIR] : 0.0003921       
                                          
                  Kappa : 0.4397          
                                          
 Mcnemar's Test P-Value : 0.4237108       
                                          
            Sensitivity : 0.7561          
            Specificity : 0.6875          
         Pos Pred Value : 0.6739          
         Neg Pred Value : 0.7674          
             Prevalence : 0.4607          
         Detection Rate : 0.3483          
   Detection Prevalence : 0.5169          
      Balanced Accuracy : 0.7218          
                                          
       'Positive' Class : 1               
                                    

In [144]:
#matrix de confusion podada
cm_table_pruned <- as.data.frame(table(Predicted = y_pred_pruned, Actual = test_data$y))

cm_table_pruned$Predicted <- factor(cm_table_pruned$Predicted, levels = c("0", "1"), 
                                    labels = c("Does not have HD", "Has HD"))
cm_table_pruned$Actual <- factor(cm_table_pruned$Actual, levels = c("0", "1"), 
                                 labels = c("Does not have HD", "Has HD"))

p3 <-ggplot(cm_table_pruned, aes(x = Actual, y = Predicted, fill = Freq)) +
  geom_tile(color = "white") +
  geom_text(aes(label = Freq), size = 8, color = "white") +
  scale_fill_gradient(low = "lightblue", high = "darkblue") +
  labs(title = paste("Confusion Matrix - Pruned Tree (α =", round(best_alpha, 6), ")"),
       x = "Actual",
       y = "Predicted") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1),
        plot.title = element_text(hjust = 0.5))

# guardo la matrix de confusion
ggsave(
  filename = paste0(output_dir, "/confusion_matrix_despues_pruning.png"),
  plot = p3,  
  width = 8,
  height = 6,
  dpi = 300
)

Interpretación:
El arbol se redujo bastante con el alpha optimo hallado y es más facil interpretar y explicar. Ademas la matrix de confusion mejoro ligeramente los valores correctos estimados (columna secundaria en azul) pero considerando que se logró podar el arbol bastante y el poder predictivo no varió mucho, es un gran herramienta lo de haber podado endógenamente.